In [ ]:
# =======================
# STEP 1: Load & Explore Resumes Dataset
# =======================

import pandas as pd

# Load all CSV files
people_df = pd.read_csv("01_people.csv")
abilities_df = pd.read_csv("02_abilities.csv")
education_df = pd.read_csv("03_education.csv")
experience_df = pd.read_csv("04_experience.csv")
person_skills_df = pd.read_csv("05_person_skills.csv")
skills_df = pd.read_csv("06_skills.csv")

# -----------------------
# 1. Show structure (columns)
# -----------------------
print("People Columns:", people_df.columns.tolist())
print("Education Columns:", education_df.columns.tolist())
print("Experience Columns:", experience_df.columns.tolist())
print("Person-Skills Columns:", person_skills_df.columns.tolist())
print("Skills Columns:", skills_df.columns.tolist())

# -----------------------
# 2. Preview samples
# -----------------------
print("\nSample People Data:")
display(people_df.head(3))

print("\nSample Education Data:")
display(education_df.head(3))

print("\nSample Experience Data:")
display(experience_df.head(3))

print("\nSample Person-Skills Data:")
display(person_skills_df.head(3))

print("\nSample Skills Data:")
display(skills_df.head(3))

# -----------------------
# 3. Quick missing values check
# -----------------------
for name, df in {
    "People": people_df,
    "Education": education_df,
    "Experience": experience_df,
    "Person-Skills": person_skills_df,
    "Skills": skills_df
}.items():
    print(f"\n{name} Missing Values:")
    print(df.isnull().sum())

# -----------------------
# 4. Ensure correct datatypes for dates
# -----------------------
education_df["start_date"] = pd.to_datetime(education_df["start_date"], errors="coerce")
experience_df["start_date"] = pd.to_datetime(experience_df["start_date"], errors="coerce")
experience_df["end_date"] = pd.to_datetime(experience_df["end_date"], errors="coerce")

print("\nData types adjusted for date fields in Education & Experience.")


People Columns: ['person_id', 'name', 'email', 'phone', 'linkedin']
Education Columns: ['person_id', 'institution', 'program', 'start_date', 'location']
Experience Columns: ['person_id', 'title', 'firm', 'start_date', 'end_date', 'location']
Person-Skills Columns: ['person_id', 'skill']
Skills Columns: ['skill']

Sample People Data:


,person_id,name,email,phone,linkedin
0,1,Database Administrator,NaN,NaN,NaN
1,2,Database Administrator,NaN,NaN,NaN
2,3,Oracle Database Administrator,NaN,NaN,NaN



Sample Education Data:


,person_id,institution,program,start_date,location
0,1,Lead City University,Bachelor of Science,07/2013,NaN
1,2,lagos state university,bsc in computer science,NaN,"Lagos, GU"
2,3,"JNTU - Kakinada, Andhra Pradesh",Master of Computer Applications in Science and...,2013,"Kakinada, Andhra Pradesh"



Sample Experience Data:


...

,person_id,skill
0,1,Database administration
1,1,Database
2,1,Ms sql server



Sample Skills Data:


,skill
0,Mongo DB-3.2
1,JNDI LDAP
2,Stored Procedures



People Missing Values:
person_id        0
name           114
email        53340
phone        53100
linkedin     46395
dtype: int64

Education Missing Values:
person_id          0
institution     1569
program         7761
start_date     21129
location       23256
dtype: int64

Experience Missing Values:
person_id         0
title           111
firm           4203
start_date     2262
end_date       2724
location      53055
dtype: int64

Person-Skills Missing Values:
person_id    0
skill        9
dtype: int64

Skills Missing Values:
skill    2
dtype: int64

Data types adjusted for date fields in Education & Experience.


/tmp/ipython-input-1021000754.py:58: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  education_df["start_date"] = pd.to_datetime(education_df["start_date"], errors="coerce")
/tmp/ipython-input-1021000754.py:59: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  experience_df["start_date"] = pd.to_datetime(experience_df["start_date"], errors="coerce")
/tmp/ipython-input-1021000754.py:60: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  experience_df["end_date"] = pd.to_datetime(experience_df["end_date"], errors="coerce")


In [ ]:
# =======================
# STEP 2: Merge & Clean Skills
# =======================

# Normalize skills first (for consistent matching)
person_skills_df['skill'] = person_skills_df['skill'].str.lower().str.strip()
skills_df['skill'] = skills_df['skill'].str.lower().str.strip()

# Merge with master skills list (to validate skills exist)
# Keep only skills found in the master list
person_skills_df = person_skills_df.merge(skills_df, on='skill', how='inner')

# Drop duplicate (person_id, skill) pairs
person_skills_df = person_skills_df.drop_duplicates(subset=['person_id', 'skill'])

print("✅ Cleaned Person-Skills shape:", person_skills_df.shape)
print("✅ Example cleaned skills:")
display(person_skills_df.head(10))


✅ Cleaned Person-Skills shape: (1873896, 2)
✅ Example cleaned skills:


,person_id,skill
0,1,database administration
4,1,database
7,1,ms sql server
17,1,ms sql server 2005
21,1,sql server
28,1,sql server 2005
33,1,sql server 2008
38,1,sql server 2008 r2
44,1,sql server 2012
49,1,sql


In [ ]:
# =======================
# STEP 3: Create Sample Parsed Resumes
# =======================

import json
import pandas as pd

# Select a few person IDs (first 3 for demo)
sample_ids = people_df['person_id'].head(3).tolist()
parsed_resumes = []

for pid in sample_ids:
    # Get person row
    person = people_df[people_df['person_id'] == pid].iloc[0]

    # --- Skills ---
    skills = person_skills_df[person_skills_df['person_id'] == pid]['skill'].tolist()
    skills = sorted(set([s for s in skills if pd.notna(s)]))  # deduplicate + clean NaN

    # --- Education ---
    edu = education_df[education_df['person_id'] == pid][
        ['institution', 'program', 'start_date', 'location']
    ].to_dict(orient='records')

    # Normalize dates in education and replace NaT with None
    for e in edu:
        if pd.notna(e['start_date']):
            e['start_date'] = str(e['start_date'])[:7]  # keep YYYY-MM format
        else:
            e['start_date'] = None

    # --- Experience ---
    exp = experience_df[experience_df['person_id'] == pid][
        ['firm', 'title', 'start_date', 'end_date', 'location']
    ].to_dict(orient='records')

    # Normalize dates in experience and replace NaT with None
    for ex in exp:
        if pd.notna(ex['start_date']):
            ex['start_date'] = str(ex['start_date'])[:7]
        else:
            ex['start_date'] = None
        if pd.notna(ex['end_date']):
            ex['end_date'] = str(ex['end_date'])[:7]
        else:
            ex['end_date'] = None

    # --- Abilities ---
    abilities = []
    if 'ability' in abilities_df.columns:
        abilities = abilities_df[abilities_df['person_id'] == pid]['ability'].dropna().tolist()

    # --- Assemble resume ---
    resume_dict = {
        'name': str(person['name']) if pd.notna(person['name']) else "",
        'email': str(person['email']) if pd.notna(person['email']) else "",
        'skills': skills,
        'education': edu,
        'experience': exp,
        'abilities': abilities
    }

    parsed_resumes.append(resume_dict)

# View sample parsed resumes
print(json.dumps(parsed_resumes, indent=2, ensure_ascii=False))

[
  {
    "name": "Database Administrator",
    "email": "",
    "skills": [
      "backups",
      "clustering",
      "database",
      "database administration",
      "maintenance",
      "ms sql server",
      "ms sql server 2005",
      "problem solving",
      "r2",
      "shipping",
      "sql",
      "sql queries",
      "sql server",
      "sql server 2005",
      "sql server 2008",
      "sql server 2008 r2",
      "sql server 2012",
      "stored procedures",
      "t-sql",
      "virtualization"
    ],
    "education": [
      {
        "institution": "Lead City University",
        "program": "Bachelor of Science",
        "start_date": "2013-07",
        "location": NaN
      }
    ],
    "experience": [
      {
        "firm": "Family Private Care LLC",
        "title": "Database Administrator",
        "start_date": "2017-04",
        "end_date": null,
        "location": "Roswell, GA"
      },
      {
        "firm": "Incomm",
        "title": "Database Administrator"

In [ ]:
# =======================
# STEP 4: Load & Clean Job Roles Dataset
# =======================

import pandas as pd

# Load Job Roles dataset with encoding fallback
try:
    jd_df = pd.read_csv("/content/IT_Job_Roles_Skills.csv", encoding="utf-8")
except UnicodeDecodeError:
    jd_df = pd.read_csv("/content/IT_Job_Roles_Skills.csv", encoding="latin-1")

# -----------------------
# 1. Show structure
# -----------------------
print("Columns available:", jd_df.columns.tolist())
print("\nRaw sample:")
display(jd_df.head(3))

# -----------------------
# 2. Normalize text
# -----------------------
jd_df['Skills'] = jd_df['Skills'].str.lower().str.strip()
jd_df['Certifications'] = jd_df['Certifications'].str.lower().str.strip()

# -----------------------
# 3. Split skills/certs into lists
# -----------------------
jd_df['Skills'] = jd_df['Skills'].apply(
    lambda x: [s.strip() for s in str(x).split(',')] if pd.notna(x) else []
)
jd_df['Certifications'] = jd_df['Certifications'].apply(
    lambda x: [c.strip() for c in str(x).split(',')] if pd.notna(x) else []
)

# -----------------------
# 4. Preview cleaned data
# -----------------------
print("\n✅ Cleaned Job Roles data sample:")
display(jd_df.head(3))


Columns available: ['Job Title', 'Job Description', 'Skills', 'Certifications']

Raw sample:


,Job Title,Job Description,Skills,Certifications
0,Admin Big Data,Responsible for managing and overseeing big da...,"Hadoop, Spark, MapReduce, Data Lakes, Data War...","Cloudera Certified Professional (CCP), Hortonw..."
1,Ansible Operations Engineer,Focuses on automating IT processes using Ansib...,"Ansible, Linux, Automation, Cloud Platforms, C...",Red Hat Certified Specialist in Ansible Automa...
2,Artifactory Administrator,Manages the Artifactory repository for build a...,"Artifactory, CI/CD, Jenkins, Docker, Maven, Gr...","JFrog Artifactory Certification, DevOps Instit..."



✅ Cleaned Job Roles data sample:


,Job Title,Job Description,Skills,Certifications
0,Admin Big Data,Responsible for managing and overseeing big da...,"[hadoop, spark, mapreduce, data lakes, data wa...","[cloudera certified professional (ccp), horton..."
1,Ansible Operations Engineer,Focuses on automating IT processes using Ansib...,"[ansible, linux, automation, cloud platforms, ...",[red hat certified specialist in ansible autom...
2,Artifactory Administrator,Manages the Artifactory repository for build a...,"[artifactory, ci/cd, jenkins, docker, maven, g...","[jfrog artifactory certification, devops insti..."


In [ ]:
# =======================
# STEP 5: Clean & Normalize JD Skills for Matching
# =======================

import re

# Ensure column names are lowercase with underscores
jd_df.rename(columns=lambda x: x.strip().lower().replace(' ', '_'), inplace=True)

# Define important columns
skill_col = 'skills'
role_col = 'job_title'

# Drop rows with missing job titles or skills
jd_df = jd_df.dropna(subset=[skill_col, role_col])

# Function to clean and normalize skills
def clean_skills(skill_str):
    # Split on commas, strip whitespace, remove stray quotes/brackets, keep multi-word intact
    skills = [
        re.sub(r"[\[\]'\" ]", "", s).lower().strip()
        for s in str(skill_str).split(',')
        if s.strip()
    ]
    return sorted(set(skills))  # deduplicate + sort

# Apply cleaning to JD skills
jd_df[skill_col] = jd_df[skill_col].apply(clean_skills)

# Add skill count column (useful for similarity calculation later)
jd_df['skill_count'] = jd_df[skill_col].apply(len)

# Remove roles with empty skill sets
jd_df = jd_df[jd_df[skill_col].map(len) > 0].reset_index(drop=True)

# Optional: sample subset for quick testing
sample_jds = jd_df[[role_col, skill_col, 'skill_count']].head(15).reset_index(drop=True)

# -----------------------
# Preview cleaned roles
# -----------------------
for index, row in sample_jds.iterrows():
    print(f"\n🔹 Role: {row[role_col]}")
    print(f"   ➤ Skills ({row['skill_count']}): {row[skill_col]}")



🔹 Role: Admin Big Data
   ➤ Skills (14): ['aws', 'azure', 'bigdataarchitecture', 'cloudcomputing', 'datagovernance', 'datalakes', 'datamodeling', 'datawarehousing', 'etl', 'hadoop', 'mapreduce', 'nosql', 'security', 'spark']

🔹 Role: Ansible Operations Engineer
   ➤ Skills (13): ['ansible', 'automation', 'aws', 'ci/cd', 'cloudplatforms', 'docker', 'infrastructureascode', 'kubernetes', 'linux', 'networking', 'python', 'scripting', 'terraform']

🔹 Role: Artifactory Administrator
   ➤ Skills (12): ['artifactory', 'buildautomation', 'ci/cd', 'cloudcomputing', 'docker', 'git', 'gradle', 'jenkins', 'jfrogcli', 'linux', 'maven', 'versioncontrol']

🔹 Role: Artificial Intelligence / Machine Learning Leader
   ➤ Skills (10): ['aistrategy', 'budgeting', 'cloudcomputing', 'communication', 'datascience', 'deeplearning', 'machinelearning', 'projectmanagement', 'stakeholdermanagement', 'teammanagement']

🔹 Role: Artificial Intelligence / Machine Learning Sr.Leader
   ➤ Skills (12): ['aistrategy', 'b

In [ ]:
# =======================
# STEP 7: Resume-to-Job Role Matching (with Skill Gap Analysis)
# =======================

def jaccard_similarity(list1, list2):
    set1, set2 = set(list1), set(list2)
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

# Normalize resume skills (same as JD cleaning)
def normalize_skills(skills):
    return [s.strip().lower() for s in skills if s and isinstance(s, str)]

# Iterate over parsed resumes
for resume in parsed_resumes:
    print(f"\n📄 Resume: {resume['name']}")
    resume_skills = normalize_skills(resume['skills'])

    role_scores = []
    for _, row in sample_jds.iterrows():
        role = row[role_col]
        jd_skills = row[skill_col]

        score = jaccard_similarity(resume_skills, jd_skills)
        if score > 0:  # skip roles with no overlap
            matched = set(jd_skills).intersection(resume_skills)
            missing = set(jd_skills) - set(resume_skills)
            role_scores.append((role, score, matched, missing))

    # Sort by similarity score
    top_roles = sorted(role_scores, key=lambda x: x[1], reverse=True)[:3]

    # Display results
    print("🎯 Top Job Role Matches:")
    for role, score, matched, missing in top_roles:
        print(f" - {role} ({round(score * 100, 2)}% match)")
        print(f"    ✔ Matched Skills: {sorted(matched)}")
        print(f"    ❌ Missing Skills: {sorted(missing)}")



📄 Resume: Database Administrator
🎯 Top Job Role Matches:
 - Big Data Specialist (3.33% match)
    ✔ Matched Skills: ['sql']
    ❌ Missing Skills: ['cloudcomputing', 'dataanalysis', 'datamining', 'datamodeling', 'datavisualization', 'etl', 'hadoop', 'nosql', 'spark', 'statisticalanalysis']
 - Data Architect (3.33% match)
    ✔ Matched Skills: ['sql']
    ❌ Missing Skills: ['aws', 'azure', 'cloudcomputing', 'datagovernance', 'dataintegration', 'datamodeling', 'datasecurity', 'datawarehousing', 'etl', 'nosql']
 - Big Data Engineer (3.23% match)
    ✔ Matched Skills: ['sql']
    ❌ Missing Skills: ['cloudcomputing', 'datapipelines', 'datawarehousing', 'etl', 'hadoop', 'java', 'kafka', 'nosql', 'python', 'scala', 'spark']

📄 Resume: Database Administrator
🎯 Top Job Role Matches:
 - Big Data Specialist (3.7% match)
    ✔ Matched Skills: ['sql']
    ❌ Missing Skills: ['cloudcomputing', 'dataanalysis', 'datamining', 'datamodeling', 'datavisualization', 'etl', 'hadoop', 'nosql', 'spark', 'stati

In [ ]:
# Re-run this if needed to ensure resume_skills and sample_jds are loaded
def skill_gap_analysis(resume_skills, required_skills):
    resume_set = set([s.lower().strip() for s in resume_skills])
    required_set = set([s.lower().strip() for s in required_skills])

    matched = sorted(list(resume_set.intersection(required_set)))
    missing = sorted(list(required_set.difference(resume_set)))

    return matched, missing

# Loop through each resume and its top 3 matches
for resume in parsed_resumes:
    print(f"\n📄 Resume: {resume['name']}")
    resume_skills = [s.lower().strip() for s in resume['skills']]

    role_scores = []
    for _, row in sample_jds.iterrows():
        role = row[role_col]
        required_skills = row[skill_col]
        score = jaccard_similarity(resume_skills, required_skills)
        role_scores.append((role, score, required_skills))

    # Top 3 roles
    top_roles = sorted(role_scores, key=lambda x: x[1], reverse=True)[:3]

    for role, score, required_skills in top_roles:
        matched, missing = skill_gap_analysis(resume_skills, required_skills)

        print(f"\n🔹 Role: {role} ({round(score * 100, 2)}% match)")
        print(f"✅ Matched Skills ({len(matched)}): {matched}")
        print(f"❌ Missing Skills ({len(missing)}): {missing}")



📄 Resume: Database Administrator

🔹 Role: Big Data Specialist (3.33% match)
✅ Matched Skills (1): ['sql']
❌ Missing Skills (10): ['cloudcomputing', 'dataanalysis', 'datamining', 'datamodeling', 'datavisualization', 'etl', 'hadoop', 'nosql', 'spark', 'statisticalanalysis']

🔹 Role: Data Architect (3.33% match)
✅ Matched Skills (1): ['sql']
❌ Missing Skills (10): ['aws', 'azure', 'cloudcomputing', 'datagovernance', 'dataintegration', 'datamodeling', 'datasecurity', 'datawarehousing', 'etl', 'nosql']

🔹 Role: Big Data Engineer (3.23% match)
✅ Matched Skills (1): ['sql']
❌ Missing Skills (11): ['cloudcomputing', 'datapipelines', 'datawarehousing', 'etl', 'hadoop', 'java', 'kafka', 'nosql', 'python', 'scala', 'spark']

📄 Resume: Database Administrator

🔹 Role: Big Data Specialist (3.7% match)
✅ Matched Skills (1): ['sql']
❌ Missing Skills (10): ['cloudcomputing', 'dataanalysis', 'datamining', 'datamodeling', 'datavisualization', 'etl', 'hadoop', 'nosql', 'spark', 'statisticalanalysis']

🔹 

In [ ]:
# =======================
# STEP 9: Skill Gap → Course Recommendations
# =======================

# Dummy course catalog (can be replaced with Coursera/edX dataset)
course_catalog = [
    {'skill': 'python', 'course_name': 'Python for Everybody', 'provider': 'Coursera', 'link': 'https://coursera.org/learn/python'},
    {'skill': 'sql', 'course_name': 'SQL for Data Science', 'provider': 'Coursera', 'link': 'https://coursera.org/learn/sql-data-science'},
    {'skill': 'power bi', 'course_name': 'Power BI Essentials', 'provider': 'Udemy', 'link': 'https://udemy.com/course/power-bi-essentials/'},
    {'skill': 'excel', 'course_name': 'Excel Skills for Business', 'provider': 'Coursera', 'link': 'https://coursera.org/learn/excel'},
    {'skill': 'tableau', 'course_name': 'Tableau A-Z', 'provider': 'Udemy', 'link': 'https://udemy.com/course/tableau-data-visualization/'},
    {'skill': 'statistics', 'course_name': 'Intro to Statistics', 'provider': 'edX', 'link': 'https://edx.org/course/statistics'}
]

# Normalize catalog skills
for course in course_catalog:
    course['skill'] = course['skill'].lower().strip()

# Build lookup dictionary
course_lookup = {c['skill']: c for c in course_catalog}

# Function: Recommend courses for missing skills
def recommend_courses(missing_skills, course_lookup):
    recs = []
    for skill in missing_skills:
        if skill in course_lookup:
            recs.append(course_lookup[skill])
    return recs

# Skill gap analysis function
def skill_gap_analysis(resume_skills, required_skills):
    resume_set = set([s.lower().strip() for s in resume_skills])
    required_set = set([s.lower().strip() for s in required_skills])

    matched = sorted(list(resume_set.intersection(required_set)))
    missing = sorted(list(required_set.difference(resume_set)))

    return matched, missing

# Loop through each resume and show role matches + recommendations
for resume in parsed_resumes:
    print(f"\n📄 Resume: {resume['name']}")
    resume_skills = [s.lower().strip() for s in resume['skills']]

    role_scores = []
    for _, row in sample_jds.iterrows():
        role = row[role_col]
        required_skills = row[skill_col]
        score = jaccard_similarity(resume_skills, required_skills)
        role_scores.append((role, score, required_skills))

    # Top 3 roles
    top_roles = sorted(role_scores, key=lambda x: x[1], reverse=True)[:3]

    for role, score, required_skills in top_roles:
        matched, missing = skill_gap_analysis(resume_skills, required_skills)

        print(f"\n🔹 Role: {role} ({round(score * 100, 2)}% match)")
        print(f"✅ Matched Skills ({len(matched)}): {matched}")
        print(f"❌ Missing Skills ({len(missing)}): {missing}")

        # Recommend courses for missing skills
        recs = recommend_courses(missing, course_lookup)
        if recs:
            print("📘 Recommended Courses:")
            for r in recs:
                print(f"   - {r['course_name']} ({r['provider']}) → {r['link']}")



📄 Resume: Database Administrator

🔹 Role: Big Data Specialist (3.33% match)
✅ Matched Skills (1): ['sql']
❌ Missing Skills (10): ['cloudcomputing', 'dataanalysis', 'datamining', 'datamodeling', 'datavisualization', 'etl', 'hadoop', 'nosql', 'spark', 'statisticalanalysis']

🔹 Role: Data Architect (3.33% match)
✅ Matched Skills (1): ['sql']
❌ Missing Skills (10): ['aws', 'azure', 'cloudcomputing', 'datagovernance', 'dataintegration', 'datamodeling', 'datasecurity', 'datawarehousing', 'etl', 'nosql']

🔹 Role: Big Data Engineer (3.23% match)
✅ Matched Skills (1): ['sql']
❌ Missing Skills (11): ['cloudcomputing', 'datapipelines', 'datawarehousing', 'etl', 'hadoop', 'java', 'kafka', 'nosql', 'python', 'scala', 'spark']
📘 Recommended Courses:
   - Python for Everybody (Coursera) → https://coursera.org/learn/python

📄 Resume: Database Administrator

🔹 Role: Big Data Specialist (3.7% match)
✅ Matched Skills (1): ['sql']
❌ Missing Skills (10): ['cloudcomputing', 'dataanalysis', 'datamining', 'd

In [ ]:
# =======================
# STEP 10: Skill Gap → Course Recommendations (with Fuzzy Matching)
# =======================

from difflib import get_close_matches

# Convert catalog to DataFrame for easy lookup
course_df = pd.DataFrame(course_catalog)
course_df['skill'] = course_df['skill'].str.lower().str.strip()

# Function to find best matching course for a skill
def find_course(skill, course_df):
    # Exact match first
    exact = course_df[course_df['skill'] == skill]
    if not exact.empty:
        return exact.to_dict('records')

    # Fuzzy match if no exact match
    all_skills = course_df['skill'].tolist()
    match = get_close_matches(skill, all_skills, n=1, cutoff=0.6)  # cutoff=0.6 is a lenient threshold
    if match:
        fuzzy = course_df[course_df['skill'] == match[0]]
        return fuzzy.to_dict('records')

    return []  # no course found

# Updated Skill Gap Loop with Fuzzy Course Recommendation
for resume in parsed_resumes:
    print(f"\n📄 Resume: {resume['name']}")
    resume_skills = [s.lower().strip() for s in resume['skills']]

    role_scores = []
    for _, row in sample_jds.iterrows():
        role = row[role_col]
        required_skills = row[skill_col]
        score = jaccard_similarity(resume_skills, required_skills)
        role_scores.append((role, score, required_skills))

    top_roles = sorted(role_scores, key=lambda x: x[1], reverse=True)[:3]

    for role, score, required_skills in top_roles:
        matched, missing = skill_gap_analysis(resume_skills, required_skills)

        print(f"\n🔹 Role: {role} ({round(score * 100, 2)}% match)")
        print(f"✅ Matched Skills: {matched}")
        print(f"❌ Missing Skills: {missing}")

        # Recommend courses
        print("🎓 Course Recommendations:")
        for skill in missing:
            courses = find_course(skill, course_df)
            if courses:
                for c in courses:
                    print(f" - {skill.title()}: {c['course_name']} ({c['provider']}) → {c['link']}")
            else:
                print(f" - {skill.title()}: No course found.")



📄 Resume: Database Administrator

🔹 Role: Big Data Specialist (3.33% match)
✅ Matched Skills: ['sql']
❌ Missing Skills: ['cloudcomputing', 'dataanalysis', 'datamining', 'datamodeling', 'datavisualization', 'etl', 'hadoop', 'nosql', 'spark', 'statisticalanalysis']
🎓 Course Recommendations:
 - Cloudcomputing: No course found.
 - Dataanalysis: No course found.
 - Datamining: No course found.
 - Datamodeling: No course found.
 - Datavisualization: No course found.
 - Etl: No course found.
 - Hadoop: No course found.
 - Nosql: SQL for Data Science (Coursera) → https://coursera.org/learn/sql-data-science
 - Spark: No course found.
 - Statisticalanalysis: Intro to Statistics (edX) → https://edx.org/course/statistics

🔹 Role: Data Architect (3.33% match)
✅ Matched Skills: ['sql']
❌ Missing Skills: ['aws', 'azure', 'cloudcomputing', 'datagovernance', 'dataintegration', 'datamodeling', 'datasecurity', 'datawarehousing', 'etl', 'nosql']
🎓 Course Recommendations:
 - Aws: No course found.
 - Azure

In [ ]:
#Step 1: Unzip and Load the Dataset
import zipfile
import os

# Unzip
with zipfile.ZipFile('/content/archive (2).zip', 'r') as zip_ref:
    zip_ref.extractall('/courses_json')

# List files to check structure
files = os.listdir('/courses_json')
print("Files in dataset:", files[:5])  # Show first 5

Files in dataset: ['processed_coursera_data.json', 'combine_preprocessing.py', 'edx_degree_programs.json', 'edx_executive_education_paidstuff.json', 'combined_dataset.json']


In [ ]:
#Step 2: import json

course_data = []

for file in files:
    if file.endswith('.json'):
        try:
            with open(f"/courses_json/{file}", 'r', encoding='utf-8') as f:
                raw = f.read().strip()
                if raw.startswith('['):
                    parsed = json.loads(raw)
                    course_data.extend(parsed)  # list of courses
                else:
                    parsed = json.loads(raw)
                    course_data.append(parsed)  # single course
        except Exception as e:
            print(f"❌ Could not parse {file}: {e}")

# Convert to DataFrame if any valid data found
if course_data:
    course_df = pd.DataFrame(course_data)
    print("✅ Loaded courses:", len(course_df))
    print(course_df.head())
else:
    print("⚠️ Still no valid courses found.")



✅ Loaded courses: 28889
                                                 url     type  \
0  https://www.coursera.org/learn/serverless-comp...   course   
1  https://www.coursera.org/learn/assist-public-s...   course   
2  https://www.coursera.org/learn/advanced-strate...   course   
3  https://www.coursera.org/learn/applying-machin...   course   
4  https://www.coursera.org/projects/automate-blo...  project   

                                         course_name  \
0   AWS Lambda إنشاء صورة مصغرة بإستخدام السيرفرل...   
1   Assisting Public Sector Decision Makers With ...   
2       Advanced Strategies for Sustainable Business   
3   Applying Machine Learning to Your Data with G...   
4          Automate Blog Advertisements with Zapier    

                       organization             instructor     rating  \
0        [Coursera Project Network]             Omar Fathy  No rating   
1          [University of Michigan]     Christopher Brooks        4.8   
2  [University of Colorado Bo

In [ ]:
course_df.columns.tolist()


['url',
 'type',
 'course_name',
 'organization',
 'instructor',
 'rating',
 'nu_reviews',
 'description',
 'skills',
 'level',
 'Duration',
 'reviews',
 'total_assignment',
 'total_app',
 'total_programming',
 'total_reading',
 'total_plugin',
 'total_ungraded',
 'total_quiz',
 'total_teammate',
 'total_peer',
 'total_discussion',
 'total_video',
 'has_assignment',
 'has_app',
 'has_programming',
 'has_reading',
 'has_plugin',
 'has_ungraded',
 'has_quiz',
 'has_teammate',
 'has_peer',
 'has_discussion',
 'has_video',
 'has_no_enrol',
 'enrollments',
 'has_rating',
 'subject',
 'has_subject',
 'provider',
 'title',
 'partner',
 'partner_keys',
 'product_source',
 'primary_description',
 'secondary_description',
 'tertiary_description',
 'tags',
 'availability',
 'language',
 'product',
 'program_type',
 'staff',
 'allowed_in',
 'blocked_in',
 'subscription_eligible',
 'subscription_prices',
 'learning_type',
 'learning_type_exp',
 'translation_languages',
 'availability_rank',
 'recen

In [ ]:
course_df[['title', 'external_url']].dropna().to_csv("/content/courses_final.csv", index=False)


In [ ]:
course_df = pd.read_csv("/content/courses_final.csv")


In [ ]:
course_df.columns = ['name', 'url']


In [ ]:
 # Step 3: Inspect and Clean Course Data
# Inspect available columns
print("🔎 Columns available:", course_df.columns.tolist())
print("\n📦 Sample Row:")
print(course_df.iloc[0])




🔎 Columns available: ['name', 'url']

📦 Sample Row:
name                    Master of Business Administration
url     https://onlinemba.ucdavis.edu/requestinfo/plp/...
Name: 0, dtype: object


In [ ]:
# Normalize column names
course_df.columns = [c.lower().strip().replace(' ', '_') for c in course_df.columns]

# Check columns after renaming
print(course_df.columns.tolist())


['name', 'url']


In [ ]:
# Step 20: Clean final course dataset (name + url only)

# Drop rows where name or url is missing
course_df = course_df.dropna(subset=['name', 'url'])

# Normalize values
course_df['name'] = course_df['name'].astype(str).str.strip()
course_df['url'] = course_df['url'].astype(str).str.strip()

# Remove duplicates if any
course_df = course_df.drop_duplicates(subset=['name', 'url']).reset_index(drop=True)

# Inspect final cleaned dataset
print("✅ Cleaned course_df shape:", course_df.shape)
print(course_df.head(5))


✅ Cleaned course_df shape: (303, 2)
                                        name  \
0          Master of Business Administration   
1  Master of Arts in International Relations   
2            Master of International Service   
3  Executive Master of Public Administration   
4          Master of Data Analytics (Online)   

                                                 url  
0  https://onlinemba.ucdavis.edu/requestinfo/plp/...  
1  https://ironline.american.edu/degrees/masters-...  
2  https://ironline.american.edu/degrees/internat...  
3  https://onlinegrad.syracuse.edu/public-adminis...  
4  http://postgradonline.sydney.edu.au/online-mas...  


In [ ]:
course_df.columns.tolist()


['name', 'url']

In [ ]:
# Step 22: Final clean + save

# Normalize column names (safe to keep)
course_df.columns = [c.lower().strip().replace(' ', '_') for c in course_df.columns]

# Drop rows where 'name' or 'url' is missing
course_df = course_df.dropna(subset=['name', 'url'])

# Normalize text
course_df['name'] = course_df['name'].astype(str).str.strip()
course_df['url'] = course_df['url'].astype(str).str.strip()

# Remove duplicates
course_df = course_df.drop_duplicates(subset=['name', 'url']).reset_index(drop=True)

# Save final dataset
course_df[['name', 'url']].to_csv("/content/courses_final.csv", index=False)

print("✅ Cleaned course dataset saved as /content/courses_final.csv")
print("📦 Shape:", course_df.shape)
print(course_df.head(5))


✅ Cleaned course dataset saved as /content/courses_final.csv
📦 Shape: (303, 2)
                                        name  \
0          Master of Business Administration   
1  Master of Arts in International Relations   
2            Master of International Service   
3  Executive Master of Public Administration   
4          Master of Data Analytics (Online)   

                                                 url  
0  https://onlinemba.ucdavis.edu/requestinfo/plp/...  
1  https://ironline.american.edu/degrees/masters-...  
2  https://ironline.american.edu/degrees/internat...  
3  https://onlinegrad.syracuse.edu/public-adminis...  
4  http://postgradonline.sydney.edu.au/online-mas...  


In [ ]:
import random
import pandas as pd

# Step 1: Load your existing course_df if not already
try:
    course_df = pd.read_csv("/content/courses_final.csv")
except Exception as e:
    print("⚠️ Error loading existing courses_final.csv:", e)
    course_df = pd.DataFrame(columns=["name", "url", "platform"])

# Initialize full_df with course_df
full_df = course_df.copy()

# Step 2: Real curated course entries for common missing tech skills
manual_courses = [
    {"name": "Meta Front-End Developer - React", "url": "https://www.coursera.org/learn/meta-front-end-developer", "platform": "Coursera"},
    {"name": "Django for Beginners", "url": "https://www.udemy.com/course/django-for-beginners/", "platform": "Udemy"},
    {"name": "REST APIs with Flask and Python", "url": "https://www.udemy.com/course/rest-api-flask-and-python/", "platform": "Udemy"},
    {"name": "Master the Coding Interview: Data Structures", "url": "https://www.udemy.com/course/master-the-coding-interview-data-structures-algorithms/", "platform": "Udemy"},
    {"name": "Docker for DevOps", "url": "https://www.udemy.com/course/docker-mastery/", "platform": "Udemy"},
    {"name": "Version Control with Git", "url": "https://www.coursera.org/learn/introduction-git-github", "platform": "Coursera"},
    {"name": "Cloud Computing Basics (AWS)", "url": "https://www.coursera.org/learn/cloud-computing-basics", "platform": "Coursera"},
    {"name": "Full-Stack Web Development with React", "url": "https://www.coursera.org/specializations/full-stack-react", "platform": "Coursera"}
]

manual_df = pd.DataFrame(manual_courses)
manual_extra_courses = [
    {"name": "Cloud Computing Basics (Coursera - LearnQuest)", "url": "https://www.coursera.org/learn/cloud-computing-basics", "platform": "Coursera"},
    {"name": "Data Mining Specialization", "url": "https://www.coursera.org/specializations/data-mining", "platform": "Coursera"},
    {"name": "Data Visualization with Tableau", "url": "https://www.coursera.org/learn/visual-analytics", "platform": "Coursera"},
    {"name": "Hadoop Platform and Application Framework", "url": "https://www.coursera.org/learn/hadoop", "platform": "Coursera"},
    {"name": "NoSQL Database Systems", "url": "https://www.edx.org/course/nosql-database-systems", "platform": "edX"},
    {"name": "Big Data Analysis with Spark", "url": "https://www.edx.org/course/big-data-analysis-with-spark", "platform": "edX"},
    {"name": "ETL and Data Pipelines with Shell, Airflow, and Kafka", "url": "https://www.coursera.org/learn/etl-data-pipelines", "platform": "Coursera"},
    {"name": "Introduction to Statistical Analysis", "url": "https://www.edx.org/course/introduction-to-statistical-analysis", "platform": "edX"},
    {"name": "Data Modeling and Relational Database Design", "url": "https://www.coursera.org/learn/data-modeling", "platform": "Coursera"},
]

manual_df2 = pd.DataFrame(manual_extra_courses)
full_df = pd.concat([full_df, manual_df2], ignore_index=True)


# Step 3: Random dummy tech skills course generator
skills = [
    "cloud computing", "data analysis", "data mining", "data modeling", "data visualization",
    "etl", "hadoop", "nosql", "spark", "statistical analysis", "machine learning", "deep learning",
    "natural language processing", "python programming", "sql", "mongodb", "big data",
    "git", "docker", "kubernetes", "devops", "tensorflow", "scikit-learn", "pandas", "numpy",
    "r programming", "ai ethics", "llms", "prompt engineering", "azure", "aws", "gcp"
]

platforms = ["Coursera", "Udemy", "EdX", "DataCamp", "Pluralsight"]

# Step 4: Generate 100 random dummy courses
extra_courses = []
for i in range(100):
    skill = random.choice(skills)
    course = {
        "name": f"Mastering {skill.title()} - Level {random.randint(1, 3)}",
        "url": f"https://example.com/{skill.replace(' ', '-')}-course-{i+1}",
        "platform": random.choice(platforms)
    }
    extra_courses.append(course)

extra_df = pd.DataFrame(extra_courses)

# Step 5: Combine and deduplicate
full_df = pd.concat([full_df, manual_df, extra_df], ignore_index=True)
full_df.drop_duplicates(subset=['name', 'url'], inplace=True)

# Step 6: Save
full_df.to_csv("/content/courses_final.csv", index=False)
print(f"✅ Final course list saved: {len(full_df)} total courses")

✅ Final course list saved: 420 total courses


In [ ]:
import re
import pandas as pd

# Reload enriched dataset
course_df = pd.read_csv("/content/courses_final.csv")

# Ensure platform column exists (fallback if missing)
if 'platform' not in course_df.columns:
    course_df['platform'] = 'Unknown'

# Mapping to fix concatenated skill names
skill_map = {
    "cloudcomputing": "cloud computing",
    "dataanalysis": "data analysis",
    "datamining": "data mining",
    "datamodeling": "data modeling",
    "datavisualization": "data visualization",
    "statisticalanalysis": "statistical analysis"
}

# Test missing skills
for raw_skill in missing:
    # Normalize skill name
    skill = skill_map.get(raw_skill, raw_skill)
    print(f"\n🔍 Looking for courses on: {skill}")

    # Regex search for skill keyword in course names
    pattern = re.compile(rf'\b{re.escape(skill)}\b', re.IGNORECASE)
    matched_courses = course_df[course_df['name'].apply(lambda x: bool(pattern.search(str(x))))]

    if not matched_courses.empty:
        for _, row in matched_courses.head(2).iterrows():
            name = row.get('name', 'Untitled')
            link = row.get('url', 'No link')
            platform = row.get('platform', 'Unknown')
            print(f" - {skill.title()}: {name} ({platform}) → {link}")
    else:
        print(f" - No course found for: {skill}")



🔍 Looking for courses on: cloud computing
 - Cloud Computing: Cloud Computing Basics (Coursera - LearnQuest) (Coursera) → https://www.coursera.org/learn/cloud-computing-basics
 - Cloud Computing: Cloud Computing Basics (AWS) (Coursera) → https://www.coursera.org/learn/cloud-computing-basics

🔍 Looking for courses on: data analysis
 - Data Analysis: Data Analysis (nan) → https://uct-online.getsmarter.com/presentations/lp/uct-data-analysis-online-short-course/
 - Data Analysis: Data Analysis for Management (nan) → https://onlinecertificatecourses.lse.ac.uk/presentations/lp/lse-data-analysis-for-management-online-certificate-course-pr/

🔍 Looking for courses on: data mining
 - Data Mining: Data Mining Specialization (Coursera) → https://www.coursera.org/specializations/data-mining
 - Data Mining: Mastering Data Mining - Level 2 (Pluralsight) → https://example.com/data-mining-course-94

🔍 Looking for courses on: data modeling
 - Data Modeling: Data Modeling and Relational Database Design 

In [ ]:
# Step 25: Derive 'skills' column from course names
# This will let us match skills against the course title text
course_df['skills'] = course_df['name'].astype(str).str.lower()

# Inspect sample
print("✅ Added 'skills' column")
print(course_df[['name', 'skills']].head(10))


✅ Added 'skills' column
                                             name  \
0               Master of Business Administration   
1       Master of Arts in International Relations   
2                 Master of International Service   
3       Executive Master of Public Administration   
4               Master of Data Analytics (Online)   
5  Master of Science in Speech-Language Pathology   
6    Bachelor of Science in Economics and Finance   
7                 Master of Health Administration   
8             Artificial Intelligence Certificate   
9           Master of Science in Computer Science   

                                           skills  
0               master of business administration  
1       master of arts in international relations  
2                 master of international service  
3       executive master of public administration  
4               master of data analytics (online)  
5  master of science in speech-language pathology  
6    bachelor of science in 

In [ ]:
course_df['skills'] = course_df['name'].str.lower()


In [ ]:
course_df['name'].sample(10).tolist()


['Mastering Aws - Level 2',
 'Mastering R Programming - Level 2',
 'Master of Science in Library and Information Science',
 'MIT SA+P Sustainable Real Estate: Analysis and Investment',
 'Advanced Project Management',
 'Organizational Leadership',
 'User Experience Design',
 'Digital Business Strategy: Harnessing Our Digital Future',
 'Economic and Political Development in Developing Countries',
 'Mastering Pandas - Level 3']

In [ ]:
import gradio as gr
import pandas as pd
import fitz  # PyMuPDF
import re
import matplotlib.pyplot as plt

# ---------- Load Job & Course Data ----------
job_df = pd.read_csv("/content/IT_Job_Roles_Skills.csv", encoding="latin-1")
job_df.rename(columns=lambda x: x.strip().lower().replace(" ", "_"), inplace=True)
job_df = job_df.dropna(subset=["skills", "job_title"])
job_df["skills"] = job_df["skills"].apply(lambda x: [s.strip().lower() for s in x.split(",")])

course_df = pd.read_csv("/content/courses_final.csv")

# ---------- Skill Extraction ----------
def extract_skills(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    words = set(text.split())

    known_skills = set()
    for skills_list in job_df["skills"]:
        if isinstance(skills_list, list):
            known_skills.update(skills_list)

    # synonym mapping
    synonym_map = {
        "github": "git",
        "rest": "rest apis",
        "restapi": "rest apis",
        "dsa": "data structures",
        "cloud": "cloud computing",
        "api": "rest apis",
    }

    matched = set()
    for word in words:
        if word in known_skills:
            matched.add(word)
        elif word in synonym_map and synonym_map[word] in known_skills:
            matched.add(synonym_map[word])

    return list(matched)

# ---------- Skill Gap Analysis ----------
def skill_gap_analysis(resume_skills, required_skills):
    resume_set = set(resume_skills)
    required_set = set(required_skills)
    matched = sorted(list(resume_set.intersection(required_set)))
    missing = sorted(list(required_set.difference(resume_set)))
    return matched, missing

# ---------- Chart Function ----------
def plot_skill_gap(matched, missing, role):
    labels = ["Matched", "Missing"]
    sizes = [len(matched), len(missing)]
    colors = ["#4CAF50", "#FF6F61"]

    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, autopct="%1.1f%%", startangle=90, colors=colors)
    ax.set_title(f"Skill Match for {role}")
    return fig

# ---------- Main Function ----------
def analyze_resume(pdf_file):
    # Extract text
    doc = fitz.open(pdf_file.name)
    resume_text = ""
    for page in doc:
        resume_text += page.get_text()

    resume_skills = extract_skills(resume_text)

    if not resume_skills:
        return "⚠️ No recognizable skills found.", None

    role_scores = []
    for _, row in job_df.iterrows():
        role = row["job_title"]
        required_skills = row["skills"]
        matched, missing = skill_gap_analysis(resume_skills, required_skills)
        score = len(matched) / len(set(required_skills)) if required_skills else 0
        role_scores.append((role, score, matched, missing))

    # Top 3 roles
    top_roles = sorted(role_scores, key=lambda x: x[1], reverse=True)[:3]

    results_text = f"✅ Extracted Resume Skills:\n{', '.join(resume_skills)}\n\n"
    plots = []

    for role, score, matched, missing in top_roles:
        results_text += f"\n🔹 Role: {role} ({round(score*100,2)}% match)\n"
        results_text += f"✔️ Matched Skills: {', '.join(matched) if matched else 'None'}\n"
        results_text += f"❌ Missing Skills: {', '.join(missing) if missing else 'None'}\n"

        # Courses
        results_text += "🎓 Recommended Courses:\n"
        for skill in missing:
            matched_courses = course_df[course_df["name"].str.contains(skill, case=False, na=False)]
            if not matched_courses.empty:
                for _, course in matched_courses.head(2).iterrows():
                    results_text += f"- {skill.title()}: {course['name']} ({course['url']})\n"
            else:
                results_text += f"- {skill.title()}: No course found.\n"

        # Generate chart for this role
        plots.append(plot_skill_gap(matched, missing, role))

    return results_text, plots[0]  # Show text + first chart for simplicity

# ---------- Gradio UI ----------
demo = gr.Interface(
    fn=analyze_resume,
    inputs=gr.File(type="filepath", label="📄 Upload Resume (PDF)"),
    outputs=[gr.Textbox(label="Analysis"), gr.Plot(label="Skill Match Chart")],
    title="🧠  SmartCareer AI ",
    description="Upload your resume and get matched job roles, missing skills, and recommended courses with visual insights."
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e58f28facda6649d5f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
